## $3.$ Hierarchical Clustering

Hierarchical clustering is an alternative to k-means methods in which allows for a rapid visualization of the clustering dynamics of the data using multiples attributes of the relationship across each observation.

This technique is widely use (in some fields even more used than k-means and other clustering algorithms) for example in the biological sciences. Similar to k-means, in hierarchical clustering there is no need to partition the data in train and test sets but we can use the power of all of the data to correctly present the hierarchical structure of the variables.

In contrast to k-means we don't need to specify the number of clusters a priori, but we can obtain a visual representation of the cluster in the for of a dendogram

![title](dendro.png)

Hierarchical clustering will be presented using the same example we have been using for unsupervised learning thanks to the tutorial provided by UC Business Analytics R Programming Guide [http://uc-r.github.io/hc_clustering](http://uc-r.github.io/hc_clustering) We will continue working with the USAarrests dataset.

In [ ]:
library(tidyverse)  # data manipulation
library(cluster)    # clustering algorithms
library(factoextra) # clustering visualization
library(dendextend) # for comparing two dendrograms

### The algorithm for hierarchical clustering can be enumerate as:

1. compute similarity matrix across elements of the matrix
2. compute cluster similarity across pairs of clusters
3. Each element is considered as a single cluster (root or bottom of tree depending on technique)
4. At the next step the cluster is divided or combined based on cluster similarity across elements of the cluster
5. Normally the split is a dichotomous split, however, there can be polytomies in some cases (especially in low variance elements)
6. Plot dendogram and cut at particular levels based on intrinsic characteristics of each cluster.

Two main techniques for hierarchical clustering AGNES (agglomerative clustering a bottom-up approach) and DIANA (Divisive hierarchical clustering a Top-down method)

We already reviewed some methods for computing similarity matrices. To compute cluster similarities we can use linkage methods that attempt to compute average (or other estimate) pairwise distances between elements of a cluster and the next cluser.

The most common methods are: complete linkage, single linkage, average linkage, centroid linkage, Ward's minimum distance.

![title](dendro_clusters.png)

In [ ]:
##Let's prepare the data

df <- USArrests
df <- na.omit(df)
df <- scale(df)
head(df)

## To generate the hierarchical clustering first we create the distance matrix using the euclidean method, then we calculate the clustering distances using the function hclust and the method complete (compere to ward)

In [ ]:
# Dissimilarity matrix
d <- dist(df, method = "euclidean")

# Hierarchical clustering using Complete Linkage
hc1 <- hclust(d, method = "complete" )

# Plot the obtained dendrogram
plot(hc1, cex = 0.6, hang = -1)

### We can also use the AGNES function. With this function you can only produce an agglomerative coefficient (measures how much cluster structure there is in the data - values closer to 1 indicate a strong clustering structure)

In [ ]:
# Compute with agnes
hc2 <- agnes(df, method = "complete")

# Agglomerative coefficient
hc2$ac
## [1] 0.8531583

### Which is the best clustering method (there is no right or wrong answer but some help to maximize the strongest clustering coefficient  

In [ ]:
# methods to assess
m <- c( "average", "single", "complete", "ward")
names(m) <- c( "average", "single", "complete", "ward")

# function to compute coefficient
ac <- function(x) {
  agnes(df, method = x)$ac
}

map_dbl(m, ac)
##   average    single  complete      ward 
## 0.7379371 0.6276128 0.8531583 0.9346210

In [ ]:
hc3 <- agnes(df, method = "ward")
pltree(hc3, cex = 0.6, hang = -1, main = "Dendrogram of agnes") 

### We can also use the function DIANA from the cluster package, however, for this clustering algorithm there is no clustering methods available. 

In [ ]:
# compute divisive hierarchical clustering
hc4 <- diana(df)

# Divise coefficient; amount of clustering structure found
hc4$dc
## [1] 0.8514345

# plot dendrogram
pltree(hc4, cex = 0.6, hang = -1, main = "Dendrogram of diana")


#### The notation for dendograms starts with the leaf that is each observation, branches are a combination of similar leaves which are combined further high on the tree until reaching the root.

Similar to k-means, We can cut our dendogram at a particular branch height which determines the number of clusters to keep.

In [ ]:
# Ward's method
hc5 <- hclust(d, method = "ward.D2" )

# Cut tree into 4 groups
sub_grp <- cutree(hc5, k = 4)

# Number of members in each cluster
table(sub_grp)

In [ ]:
##We can add the cluster identity to the original matrix
USArrests %>%
  mutate(cluster = sub_grp) %>%
  head

### For better visualization we can generate borders that separate each cluster.

In [ ]:
plot(hc5, cex = 0.6)
rect.hclust(hc5, k = 4, border = 2:5)

In [ ]:
fviz_cluster(list(data = df, cluster = sub_grp))

### We can also compare two dendrograms that were obtained using different hierarchical clusterings to compare the new and stable branches.

In [ ]:
# Compute distance matrix
res.dist <- dist(df, method = "euclidean")

# Compute 2 hierarchical clusterings
hc1 <- hclust(res.dist, method = "complete")
hc2 <- hclust(res.dist, method = "ward.D2")

# Create two dendrograms
dend1 <- as.dendrogram (hc1)
dend2 <- as.dendrogram (hc2)

tanglegram(dend1, dend2)

### The function entanglement produces a measurement of how similar the trees are (good alingment)

In [ ]:
dend_list <- dendlist(dend1, dend2)

tanglegram(dend1, dend2,
  highlight_distinct_edges = FALSE, # Turn-off dashed lines
  common_subtrees_color_lines = FALSE, # Turn-off line colors
  common_subtrees_color_branches = TRUE, # Color common branches 
  main = paste("entanglement =", round(entanglement(dend_list), 2))
  )

## We also need to determine the optimal number of clusters to cut

Similar to k-means we can use the elbow method to generate a scree plot and visualy inspect the best number of clusters.

In [ ]:
fviz_nbclust(df, FUN = hcut, method = "wss")